## First Question : scarp top 250 movie imbd with information

### import libraries

In [91]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

##### get main url

In [92]:
urlmain = 'https://www.imdb.com/chart/top/'


#### Send get request to url 

In [93]:
headers = {'Accept-Language': 'en-US',"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [94]:
response = requests.get(url=urlmain , headers=headers)

In [95]:
print(response.status_code)

200


##### parse content

In [96]:
soup = BeautifulSoup(response.content, 'html.parser')

get link for each movies

In [97]:
links_250 = soup.find_all('div', class_='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-b85248f1-7 lhgKeb cli-title')

In [98]:
scrap_info_movies = []

In [99]:
import time
session = requests.Session()
for link in links_250[:2]: 
    a_element = link.find("a")
    get_att = a_element.get('href')
    movie_urls = f"https://www.imdb.com{get_att}"
    response_each_movie = requests.get(url=movie_urls, headers=headers)
    movies_soup = BeautifulSoup(response_each_movie.content, 'html.parser')
    titles = movies_soup.find('span', class_='sc-afe43def-1 fDTGTb').text

    ul_element = movies_soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'})
    years = None
    parental_guide = None
    runtime = None

    for li_element in ul_element.find_all('li', {'class': 'ipc-inline-list__item'}):
        ainner_element = li_element.find('a', {'class': 'ipc-link ipc-link--baseAlt ipc-link--inherit-color'})
        if ainner_element:
            href = ainner_element.get('href')
            text = ainner_element.text
            if 'releaseinfo' in href:
                years = text
            elif 'parentalguide' in href:
                parental_guide = text
        else:
            runtime = li_element.text
    genre_list = movies_soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
    genre = [genre.find('span', class_='ipc-chip__text').text for genre in genre_list]
    
    scrap_info_movies.append({'title': titles, 'genres': genre, 'stars': 'stars', 'directors': 'directors', 'writers': 'writers'})
    time.sleep(2)

session.close()


In [100]:
top_250 = pd.DataFrame(scrap_info_movies)

In [101]:
top_250.head(50)

,title,genres,stars,directors,writers
0,The Shawshank Redemption,[Drama],[],[],[]
1,The Godfather,"[Crime, Drama]",[],[],[]


In [102]:
"""     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    directore = [directore.get_text() for directore in directore_list]
    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    writer = 
    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    star = 
    gross_us_canada =   """

"     directore_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    directore = [directore.get_text() for directore in directore_list]\n    writer_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    writer = \n    star_list = movies_soup.find_all('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')\n    star = \n    gross_us_canada =   "